In [18]:
!conda install -c anaconda beautifulsoup4 -y

Solving environment: done

# All requested packages already installed.



In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import bs4 as bs
from bs4 import BeautifulSoup

In [24]:
!conda install -c anaconda lxml -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda

The following packages will be UPDATED:

    lxml: 4.3.1-py36hefd8a0e_0 --> 4.5.0-py36hefd8a0e_0 anaconda


lxml-4.5.0           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [44]:
import requests
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [132]:
#get the data using beautifulsoup from the link of wikipedia
soup=BeautifulSoup(url,'lxml')
match=soup.tbody
print(match.prettify())

<tbody>
 <tr>
  <th>
   Postal code
  </th>
  <th>
   Borough
  </th>
  <th>
   Neighborhood
  </th>
 </tr>
 <tr>
  <td>
   M1A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M2A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M3A
  </td>
  <td>
   North York
  </td>
  <td>
   Parkwoods
  </td>
 </tr>
 <tr>
  <td>
   M4A
  </td>
  <td>
   North York
  </td>
  <td>
   Victoria Village
  </td>
 </tr>
 <tr>
  <td>
   M5A
  </td>
  <td>
   Downtown Toronto
  </td>
  <td>
   Regent Park / Harbourfront
  </td>
 </tr>
 <tr>
  <td>
   M6A
  </td>
  <td>
   North York
  </td>
  <td>
   Lawrence Manor / Lawrence Heights
  </td>
 </tr>
 <tr>
  <td>
   M7A
  </td>
  <td>
   Downtown Toronto
  </td>
  <td>
   Queen's Park / Ontario Provincial Government
  </td>
 </tr>
 <tr>
  <td>
   M8A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M9A
  </td>
  <td>
   Etobicoke
  </td>
  <td>
   Islington Avenue
  </td>
 </

In [133]:
import sys
sys.getsizeof(match)

56

In [230]:
#make a dataframe from the data collected from Wikipedia
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Postalcode = df["Postal code"].tolist()
Borough = df["Borough"].tolist()
Neighborhood = df["Neighborhood"].tolist()

In [231]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [232]:
#check size and type
df.dtypes
df.shape

(180, 3)

In [233]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned. -- Dropped cells not assigned 
df = df[~df['Borough'].isin(['Not assigned'])]
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [234]:
df.shape #check new size of the database

(103, 3)

In [235]:
#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, ...
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df['Neighborhood'] = df['Neighborhood'].str.replace(' /',',')
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [236]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df.loc[df['Neighborhood'] == 'NaN', 'Neighborhood'] = df['Borough'] 

In [237]:
# reset index
df = df.reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [238]:
#check the final shape of the database
df.shape

(103, 3)